## Use the words contained in a movie review to predict whether the review is positive or negative

### Create list of files of positive and negative reviews
### TODO: Try with keras flow_from_directory

In [1]:
import tensorflow as tf

filepath_dataset_pos = tf.data.Dataset.list_files('./aclImdb/train/pos/*.txt', seed=42)
filepath_dataset_neg = tf.data.Dataset.list_files('./aclImdb/train/neg/*.txt', seed=42)

for filepath in filepath_dataset_pos.take(1):
    a_filepath = filepath
    print(a_filepath)

tf.Tensor(b'./aclImdb/train/pos/8782_9.txt', shape=(), dtype=string)


In [2]:
# Get the text from the file into a dataset
dataset_pos = tf.data.TextLineDataset(filepath_dataset_pos)
dataset_neg = tf.data.TextLineDataset(filepath_dataset_neg)

for a in dataset_pos.take(1):
    print(a.numpy())
    print('\n')

b"Overall this is a delightful, light-hearted, romantic, musical comedy. I suppose a small case could be made for the movie being to long. But I'm not sure what you would cut out. The singing that Kelly and Sinatra do? No. The fabulous dancing that Kelly does? No. The time the movie takes to develop the story line and develop the relationships of the characters? No (that seems to be a common complaint many times that more recent movies don't develop the characters).<br /><br />Some comment that Iturbi didn't bring much to the movie but this gives us a chance to see and hear a great talent from the 1040s. So what if he wasn't an actor? He was an important part of the movie as the basic plot was to get Grayson an audition with him. <br /><br />Originally Katherine Grayson wanted to be an opera star. Louis B. Mayer brought her to MGM for a screen test that included an aria. During her audition in the movie there is a shot of the MGM brass nodding and smiling. You can just imagine it was l

In [3]:
# TODO: Learn Regex!

def preprocess(line):
    """ Take a single review and process it to output a list of words 
    """
    
    line = tf.strings.lower(line)
    line = tf.strings.regex_replace(line, pattern='<br', rewrite=' ')
    line = tf.strings.regex_replace(line, pattern='/>', rewrite=' ')
    line = tf.strings.regex_replace(line, pattern='\!', rewrite=' ')
    line = tf.strings.regex_replace(line, pattern='\.', rewrite=' ')
    line = tf.strings.regex_replace(line, pattern='\,', rewrite=' ')
    line = tf.strings.regex_replace(line, pattern='\?', rewrite=' ')
    line = tf.strings.regex_replace(line, pattern='\-', rewrite=' ')
    line = tf.strings.regex_replace(line, pattern='\'', rewrite=' ')
    line = tf.strings.regex_replace(line, pattern='\"', rewrite=' ')
    line = tf.strings.regex_replace(line, pattern='\(', rewrite=' ')
    line = tf.strings.regex_replace(line, pattern='\)', rewrite=' ')
    line = tf.strings.regex_replace(line, pattern='[\d]', rewrite=' ')
    line = tf.strings.split(line, '')
    return line

# Do the above word mapping for every dataset
dataset_words_pos = dataset_pos.map(preprocess)
dataset_words_neg = dataset_neg.map(preprocess)
for data in dataset_words_pos.take(1):
    print(data.numpy()[0:10])
    

[b'i' b'have' b'seen' b'this' b'movie' b'when' b'i' b'was' b'about'
 b'years']


### Define an encoding for the words in the reviews

In [4]:
vocab = ['i', 'went', 'shopping']
indices = tf.range(len(vocab), dtype=tf.int64)
table_init = tf.lookup.KeyValueTensorInitializer(vocab, indices) # TextFileInitializer?
num_oov_buckets = 100
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)

def encode_words(words):
    return table.lookup(words)

dataset_encoded_pos = dataset_words_pos.map(encode_words)
dataset_encoded_neg = dataset_words_neg.map(encode_words)

for encoding in dataset_encoded_pos.take(3):
    print(encoding.numpy())
    print('\n')

[  0  14  22  48  83  57  80  19   0  40  65  18  55  23   0  41  41  76
 100  53  66  37  44  53  42   4  59  18 100  27  23  49  18  22  48 100
  53  41  87  28 100  28  44  70  41   0  53  88  66  41  28  80  76   0
  53  44  58  41  23  31  74  71  15  42  59  38   0  37  31  53  49  49
  82   0  37  31  84  37  45  55  90  64  41  57  93  69   4  18  18  94
  51  63  55  30 100  30  56  22  25]


[  0  24  87  21  57  45   9   8  98  91  79   4  51  76   0  53  94  40
  52  10  41   4  10  23   4  15 102  42  36 102  92   7   4  52  63  57
  79  21   4  19   8  66 100  52  88  66  91  10  44  43  22  78  24  88
  57  68  57  94  21  64  22  40  39  18   4  46  75  37  40  91  99  65
  57  10  24  31  69  44  99  92  24  69  41   4  12  80  45 100  41   4
  66  44  24 100  45  85  24  53  21  23  41  42  26  83  71  87  24 102
  41  87  24  70  41  74  83  71  37 102  41  71  25 100   4  94  71  22
  18  55]


[ 70   6 100   3  57  59  75  26  26  25  68  88  58   4  98  21  94  26

In [5]:
# add the labels into the dataset
labels_pos = tf.data.Dataset.from_tensor_slices(tf.ones((12500, 1)))
labels_neg = tf.data.Dataset.from_tensor_slices(tf.zeros((12500, 1)))

#print(len(list(dataset_encoded)))

dataset_with_labels_pos = tf.data.Dataset.zip((dataset_encoded_pos, labels_pos))
dataset_with_labels_neg = tf.data.Dataset.zip((dataset_encoded_neg, labels_neg))

for x in dataset_with_labels_pos.take(1):
    print('data\n', x[0].numpy())
    print('label\n', x[1].numpy())

data
 [ 57  41  71  84  37   4  15  65  22  48  22  23  57  48  91  11  55  60
   6  78  44  31 102   3  42  24   7  69  91  57  41  23  42  10  11  22
  23  69  11   4  22  67  64  44  41   4   3  42  64  24 100  96  22  23
  12  52   4  11 100  58  71  53  81  41  18  31  71  53  65  64  41  84
  42  72  43  53  41 100  84  92  64  57  22  19   6  53  11  10   4  44
   4  76   6  53  11  99   5 100  15  93  70  10  42  86  68   6  53  11
  91  92  17  22  23  57  11  91  23  76 100  73  25  18  67  23  11   4
  94   5   0  84  79  23  42  63  64  20  25  91  19  42  63  64  60  69
  41   4  18  42   8  49  18  30  36  18  59  98  50  74  83  81  11   8
  11  91  76  57  41  86  41  22  23  57  42  64  60  24  81  19 100  57
 102  45  43  91  24  81  65  40  45  10  85  10   4   4  84 100  10   4
   6  50  16  83  57  99  97  15  66  45  57  20  74  66  95  80 102  60
  84  94  74  44  57  83 100  15  57  92  76  23  79  66  44  84  91  79
 100  86   7   6  57 101  45  89  57  48  41 

In [6]:
# combine (and shuffle the pos and neg examples)...

full_dataset_with_labels = dataset_with_labels_neg.concatenate(dataset_with_labels_pos)
full_dataset_with_labels_shuffled = full_dataset_with_labels.shuffle(20000)

for x in full_dataset_with_labels_shuffled.take(10):
    print('data\n', x[0].numpy())
    print('label\n', x[1].numpy())

data
 [ 35   4 100  63  74  42  11  37  37  42  24  87 100  83  10  22  23  42
  31  95  11  74  58  44 100  56  15  55  55  52  86  16  41   4  42   4
  44  26  95  83   0  55 100  28  39   4 100  20  44  55  22  11   9  41
  55  29  87  18   7 100  71  55  42  91  74  42  19   6  50  22  55  22
  57  39  91   0  55 100  72  42  95  34  95   0  44  42  11   0  60  28
  61  41   4  42  85  38  93  58  86  57  23  64  23  68  84  74  24  91
  68  85  83  53  57  21  75  75  13   6  26  23  57  98  57  29  10  57
  44 100  62  11  53  57  43  42  12  80  23  55  91  83  18  33  64  57
  98   4  33   6 102  97  57  99  24  64   4  53  31  72  57  32  79  74
  26  57  70  52  89  18  41  42  15  30  60  91  10  18  42  14  64  30
  69  23  42   4  41  70  53  74   4  41   4  99  21  62  42  37  97  99
  25  42   6  90  82  18  62  48  17  55  71  83  43  46  59  40  64  38
  94  73  34  23  77  57  11  98  15  47  24   8  29  57  11  19  44 100
  68  36  25   8  41   4  55  13  40  64  81 

In [7]:
# pad the encodings so all messages are the same length

#...not certain this is requried

# def pad_encodings(encoding):
#     return tf.pad(encoding, )

## Create an embedding for the encoded reviews

In [8]:
embedding_dim=4

# TODO: How to specify input size - need to pad encodings?
# And if so need to sort out masking so padding is ignored.

model = tf.keras.Sequential([
  tf.keras.layers.Embedding(201, embedding_dim),
  tf.keras.layers.GlobalAveragePooling1D(),
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 4)           804       
_________________________________________________________________
global_average_pooling1d (Gl (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 16)                80        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 901
Trainable params: 901
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [10]:
model.fit(full_dataset_with_labels_shuffled)

25000/25000 [==============================] - 78s 3ms/step - loss: 0.6909 - accuracy: 0.5259


### Create a preprocessing layer for reading in the reviews

In [11]:
import tensorflow.keras as keras

class EncodingLayer(keras.layers.Layer):
    def adapt(self):
        pass
    def call(self):
        pass

In [12]:
encoding_layer = EncodingLayer()
encoding_layer.adapt(data_sample)

NameError: name 'data_sample' is not defined